In [1]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

# from connector.mosaicio import MosaicioConnector

In [1]:
pwd

'/notebooks/notebooks/Organization_Risk'

In [2]:
# data_ind_U  = pd.read_csv(r"/notebooks/notebooks/Synthetic_data_Clustered_individual_U.csv")
# #uncomment to run for org
# # data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

# ind_data_thresholds_U = pd.read_csv(r"/notebooks/notebooks/threshold_individual_U.csv")
# #uncomment to run for org
# # org_data_thresholds_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv")

# weight_table_ind_U = pd.read_csv(r"/notebooks/notebooks/weight_individual_U.csv")
# #uncomment to run for org
# # weight_table_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv")

In [3]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [4]:
cursor.execute('''INSERT INTO  FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_ANALYTICAL_RULES
select
A.ORGANIZATION_MASTER_ID,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end Risk_score,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG b 
on a.ORGANIZATION_MASTER_ID = b.ORGANIZATION_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO ''')
conn.commit()

IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint 'XPK_RISK_ORGANIZATION_ANALYTICAL_RULES'. Cannot insert duplicate key in object 'FINCORE_ANALYTICS.RISK_ORGANIZATION_ANALYTICAL_RULES'. The duplicate key value is (1, O002001001). (2627) (SQLExecDirectW)")

In [ ]:
risk_an_per = pd.read_sql_query('''select 
A.ORGANIZATION_MASTER_ID,
A.RULE_NO,
A.RULE_VALUE,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end THRESHOLD_FLG,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG b 
on a.ORGANIZATION_MASTER_ID = b.ORGANIZATION_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
''',conn)

In [ ]:
# for i in tqdm(range(10)):
#     df_cal = pd.read_csv((r"cal_") + str(i) + ".csv")
#     risk_an_per = risk_an_per.append(df_cal, ignore_index=True)

In [ ]:
import math
def round_func(num):
    if num-math.floor(num)<0.5:
        return math.floor(num)
    return math.ceil(num)

In [ ]:
risk_an_per.drop_duplicates(inplace=True)
risk_an_per.to_csv(r"/data/risk_an_per.csv")

In [ ]:
risk_an_per = pd.read_csv(r"/data/risk_an_per.csv")
risk_an_per.drop(columns = ['Unnamed: 0'], inplace = True)
risk_an_per.dropna(inplace =  True)

In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [ ]:
risk_an_per.drop_duplicates(inplace=True)

In [ ]:
risk_an_per

In [ ]:
# for ind,row in tqdm(risk_an_per.iterrows()):
#     conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, THRESHOLD_FLG)
#     values (?,?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RISK_SCORE']))
# #     conn.execute('''INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RULE_VALUE'],row['THRESHOLD_FLG']))
# #     conn.execute('INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)', result)
# conn.commit()
# conn.close()


In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock)", conn)

In [ ]:
# Make a list of all the Static rules

static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Organization') & (risk_lkp['RISK_TYPE'] != 'Auto High Rules ')]['RULE_NO'].to_list()
static_rules

In [ ]:
# Read risk_ORGANIZATION_summary_stg_rules for the static rules
if len(static_rules) == 1:
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(static_rules[0]),conn)
    
elif(len(static_rules)== 0):
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    static_rules = tuple(static_rules)
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (static_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [ ]:
stat_rule_df[~pd.isnull(stat_rule_df['RISK_SCORE_STATIC'])]

In [ ]:
# The stat_rule_df is converted to a data frame which matches the shape of the risk_an_per

stat_rule_df = stat_rule_df[['ORGANIZATION_MASTER_ID','RULE_NO','RISK_SCORE_STATIC']]
stat_rule_df.drop_duplicates(inplace=True)

In [ ]:
# stat_rule_df['THRESHOLD_FLG'] = 10
#Dont need to insert list based(static) rules in analytical table
stat_rule_df.rename(columns={'RISK_SCORE_STATIC':'RULE_VALUE'},inplace=True)
stat_rule_df['THRESHOLD_FLG'] = stat_rule_df['RULE_VALUE']

In [ ]:
# Appending the data of static and non static rules to calculate the final risk score

risk_an_per = risk_an_per.append(stat_rule_df)

In [ ]:
temp = risk_an_per.groupby(['ORGANIZATION_MASTER_ID','RULE_NO']).size().reset_index()

In [ ]:
risk_an_per.to_csv(r"/data/risk_an_per_with_static.csv")

In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
error = pd.read_sql_query('''select organization_master_id,rule_no,COUNT(distinct RISK_SCORE_STATIC) as no_of_scores from FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES
group by organization_master_id,rule_no
having COUNT(distinct RISK_SCORE_STATIC) > 1''',conn)
error

In [ ]:
risk_per_analytical = risk_an_per
risk_weight =  pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) order by rule_no ",conn)

#Changing the structure of the table
risk_per_analytical = risk_per_analytical.pivot(index='ORGANIZATION_MASTER_ID', columns='RULE_NO', values=['THRESHOLD_FLG'])
#col[1] contains only the first name of the column
risk_per_analytical.columns = [col[1] for col in risk_per_analytical.columns.values]
# risk_per_analytical = risk_per_analytical.reset_index()
risk_per_analytical = risk_per_analytical.reset_index()



In [ ]:
#weight columns contains all the rule columns
weight_col = risk_per_analytical.columns
weight_col = [col for col in weight_col if col.startswith('O0')]
#Risk_weight contains the weightage of the weight_col
risk_weight = risk_weight[risk_weight['RULE_NO'].isin(weight_col)]['WEIGHTAGE']
risk_weight
# risk_weight.pivot(columns='RULE_NO', values=['WEIGHTAGE'])

In [ ]:
#creating the dataframe of the weight_col to make comparison easier
rule_weight_df = pd.DataFrame(data = [risk_weight.values],columns = weight_col)
rule_weight_df

In [ ]:
weighted_avg_denom = pd.read_sql_query("select sum(WEIGHTAGE) from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) where PERSON_TYPE = 'Individual' and IS_ACTIVE_FLAG = 1 and RISK_TYPE <> 'Auto High Rules ' ",conn)

weighted_avg_denom = weighted_avg_denom.iloc[0,0]
weighted_avg_denom

In [ ]:
risk_ORGANIZATION_summary_stg_rules = pd.read_sql_query("Select * from fincore_db.fincore_analytics.risk_ORGANIZATION_summary_stg_rules with (nolock)",conn)

weights = pd.read_sql_query("Select RULE_NO,WEIGHTAGE from fincore_db.fincore_analytics.risk_lkp_rules with (nolock)",conn)

risk_ORGANIZATION_summary_stg_rules = risk_ORGANIZATION_summary_stg_rules.merge(weights,how = 'inner',on = 'RULE_NO')

risk_ORGANIZATION_summary_stg_rules.drop(columns=  ['RULE_VALUE','RISK_SCORE_STATIC','CREATE_DATE'],inplace = True)

risk_ORGANIZATION_summary_stg_rules.drop_duplicates(inplace = True)

risk_ORGANIZATION_summary_stg_rules.drop(columns= ['RULE_NO'],inplace = True)

denominator = risk_ORGANIZATION_summary_stg_rules.groupby('ORGANIZATION_MASTER_ID').sum()

risk_per_analytical = risk_per_analytical.merge(denominator,how = 'inner',on = 'ORGANIZATION_MASTER_ID')

In [ ]:
risk_per_analytical

In [ ]:
# #here calc risk score
# per_analytical_cols = risk_per_analytical.columns.to_list()
# #per_analytical will contain all the columns of risk_per_analytical in addition to below columns
# per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
# per_analytical = []
# count = 1
# for ind,row in tqdm(risk_per_analytical.iterrows()):
#     if count%10000 == 0:
#         print(row['PERSON_MASTER_ID'],' HERE')
#     count += 1
#     l1 = []

#     if False: 
# #     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
#         data = row[weight_col]
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values)
#         rule_list = "None" #rule_list will be empty in this case
#         l1.append(rule_list)
#         risk_score = -1 #nan risk score
#         l1.append(risk_score)
# #         print(row)
# #         print(risk_score)
# #         print(rule_list)
# #         l1 containg the entire row including the risk score and appending it with per_analytical list
#         per_analytical.append(l1)
# #         break
#     else:
        
#         data = row[weight_col] * rule_weight_df 
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values[0])
        
#         #Rule_list will include only those columns that has non null values
#         rule_list = [val for val in row.keys() if val.startswith('P0') and ~np.isnan(row[val]) and row[val] > 0 ]
#         rule_list = "_".join(rule_list)
#         num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
#         l1.append(rule_list)
#         #calculating risk score by summing the row and dividing by count of non null rules
# #         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
#         risk_score = (np.sum(num_rules))/(weighted_avg_denom)
#         l1.append(risk_score)
#         per_analytical.append(l1)
#         print(l1)
        

In [ ]:
risk_per_analytical

In [ ]:
#here calc risk score
per_analytical_cols = risk_per_analytical.columns.to_list()
#per_analytical will contain all the columns of risk_per_analytical in addition to below columns
per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
per_analytical_cols.remove('WEIGHTAGE')
per_analytical = []
count = 1
for ind,row in tqdm(risk_per_analytical.iterrows()):
    if count%10000 == 0:
        print(row['ORGANIZATION_MASTER_ID'],' HERE')
    count += 1
    l1 = []

    if False: 
#     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
        data = row[weight_col]
        l1.append(row['ORGANIZATION_MASTER_ID'])
        l1.extend(data.values)
        rule_list = "None" #rule_list will be empty in this case
        l1.append(rule_list)
        risk_score = -1 #nan risk score
        l1.append(risk_score)
#         print(row)
#         print(risk_score)
#         print(rule_list)
#         l1 containg the entire row including the risk score and appending it with per_analytical list
        per_analytical.append(l1)
#         break
    else:
        
        data = row[weight_col] * rule_weight_df 
        l1.append(row['ORGANIZATION_MASTER_ID'])
        l1.extend(data.values[0])
        
        #Rule_list will include only those columns that has non null values
        rule_list = [val for val in row.keys() if val.startswith('O0') and ~np.isnan(row[val]) and row[val] > 0 ]
        rule_list = "_".join(rule_list)
        num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
        l1.append(rule_list)
        #calculating risk score by summing the row and dividing by count of non null rules
#         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
        risk_score = (np.sum(num_rules))/row['WEIGHTAGE']
        l1.append(risk_score)
        per_analytical.append(l1)
        print(l1)
        

In [ ]:
data_ind_U = pd.read_sql_query('''select ORGANIZATION_MASTER_ID,PEER_GROUP_ID from fincore_db.fincore_analytics.risk_ORGANIZATION_summary_stg with (nolock) ''',conn)

In [ ]:
#creating a new dataframe
risk_per_analytical_modified = pd.DataFrame(data = per_analytical,columns=per_analytical_cols)

In [ ]:
#Merging it with data_ind_U to get the peer_group_id
risk_per_analytical_modified_df = pd.merge(data_ind_U[['ORGANIZATION_MASTER_ID','PEER_GROUP_ID']],risk_per_analytical_modified,on = 'ORGANIZATION_MASTER_ID')

In [ ]:
risk_per_analytical_modified_df.drop_duplicates(inplace=True)

In [ ]:
data = risk_per_analytical_modified_df[['ORGANIZATION_MASTER_ID','PEER_GROUP_ID','RISK_SCORE','RULE_LIST']]
data.to_csv(r"/data/risk_organization.csv")

In [ ]:
# data = pd.read_csv(r"/data/risk_person.csv")

In [ ]:
try:
    data.drop(columns = ['Unnamed: 0'],axis = 1,inplace= True)
except:
    pass
    

In [ ]:
data['RISK_SCORE'].fillna(0,inplace=True)

In [ ]:
data['RULE_LIST'].fillna('None',inplace=True)

data.to_csv(r"/data/risk_organization_final.csv")

In [ ]:
data.head()

In [ ]:
data['RISK_SCORE'] = data['RISK_SCORE'].apply(lambda x: round_func(x)) 

In [ ]:
auto_high_df = pd.read_sql_query('''Select ORGANIZATION_MASTER_ID from fincore_db.fincore_analytics.RISK_ORGANIZATION_SUMMARY_STG_RULES A inner join fincore_db.fincore_analytics.RISK_LKP_RULES  B ON A.RULE_NO =  B.RULE_NO WHERE RISK_TYPE = 'Auto High Rules ' ''',conn)
auto_high = auto_high_df['ORGANIZATION_MASTER_ID'].to_list()
data = data[~data['ORGANIZATION_MASTER_ID'].isin(auto_high)]

In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# conn = pyodbc.connect(config_data["connection"])
count = 1
#UPDATING RISK_ORGANIZATION
# for ind,row in risk_per_analytical_modified_df.iterrows():
for ind,row in tqdm(data.iterrows()):
    if count%10000 == 0:
        print(row['ORGANIZATION_MASTER_ID'],' HERE')
        conn.commit()
    count += 1
    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION (ORGANIZATION_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RISK_RULE_LIST,CREATE_DATE)
    values ({},'{}',{},'{}',{})'''.format(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],row['RULE_LIST'],'Getdate()'))
    
#     conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_ORGANIZATION (ORGANIZATION_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RISK_RULE_LIST)
#     values (?,?,?,?)''',(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],row['RULE_LIST']))

conn.commit()
conn.close()

In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
risk_person =  pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION] with (nolock)",conn)

In [ ]:
%store -r no_of_records

In [ ]:
no_of_records

In [ ]:
%store -r job_id

job_id

In [ ]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()


In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success',NO_OF_RECORD = {no_of_records} where  JOB_RUN_ID ={job_id}''')

cursor.commit()
conn.close()